<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Importing Libraries</b>

In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime
import statistics
from sklearn.preprocessing import OneHotEncoder

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Defining Helper Functions</b>

In [2]:
def IQROutlierCheck(df, col):
    
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q1 + 1.5 * IQR
    critic_score_outliers = df[(df[col] < lower) | (df[col] > upper)]
    
    return critic_score_outliers

In [3]:
def OutliersInfo(df, cols):

    outlier_dict=dict()
    for col in cols: 
        print(f"\n{col}")
        print("-"*35)
        critic_outliers = IQROutlierCheck(df, col)
        outlier_dict[col] = critic_outliers.index
        print(f"Number of outlier samples produced by IQR is {critic_outliers[col].shape[0]}")
        for i in range(0, 2):

            print("{}% percentile value is {:3.3f}".format(i, np.percentile(df[col], i)))
        for i in range(98, 101):
            print("{}% percentile value is {:3.3f}".format(i, np.percentile(df[col], i)))

        percent = np.percentile(df[col], 1)
        count = len(df[df[col]<percent])    
        print(f"\n\t- Number of values less than {percent} is {count}")
        percent = np.percentile(df[col], 99)
        count = len(df[df[col]>percent]) 
        print(f"\t- Number of values greater than {percent} is {count}")

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Reading and Combining Transactions Datasets</b>

In [4]:
# Define data types for each column to reduce memory usage
dtypes = {
    'card_id': 'category',
    'merchant_id': 'category',
    'month_lag': 'int8',
    'authorized_flag': 'category',
    'category_1': 'category',
    'installments': 'int16',
    'purchase_amount': 'float32',
    'city_id': 'int16',
    'state_id': 'int8',
    'subsector_id': 'int8',
    'merchant_category_id': 'int16',
    'category_2': 'float32',  
    'category_3': 'category'
}


In [5]:
# Define chunk size
chunk_size = 500000

# Initialize lists to store chunks
history_chunks = []
new_merchant_chunks = []

# Read historical transactions in chunks
for chunk in pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/historical_transactions.csv', chunksize=chunk_size, dtype=dtypes):
    history_chunks.append(chunk)

# Read new merchant transactions in chunks
for chunk in pd.read_csv('/kaggle/input/elo-merchant-category-recommendation/new_merchant_transactions.csv', chunksize=chunk_size, dtype=dtypes):
    new_merchant_chunks.append(chunk)

# Combine all chunks from both datasets
combined_transactions = pd.concat(history_chunks + new_merchant_chunks, axis=0)

# Clean up memory
del history_chunks, new_merchant_chunks
gc.collect()

print(combined_transactions.shape)

(31075392, 14)


The combination of the two transaction datasets, 'historical_transactions' and 'new_merchant_transactions', likely happened to create a comprehensive view of all transactions for analysis, This unified dataset would allow for a more complete understanding of user behavior and spending patterns.

In [6]:
# Displaying the first 5 rows of the combined transactions

combined_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [8]:
# Summary of combined transactions DataFrame including data types, non-null counts, and memory usage

combined_transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31075392 entries, 0 to 1963030
Data columns (total 14 columns):
 #   Column                Dtype   
---  ------                -----   
 0   authorized_flag       object  
 1   card_id               object  
 2   city_id               int16   
 3   category_1            category
 4   installments          int16   
 5   category_3            category
 6   merchant_category_id  int16   
 7   merchant_id           object  
 8   month_lag             int8    
 9   purchase_amount       float32 
 10  purchase_date         object  
 11  category_2            float32 
 12  state_id              int8    
 13  subsector_id          int8    
dtypes: category(2), float32(2), int16(3), int8(3), object(4)
memory usage: 1.7+ GB


<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:180%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:800;
            text-align:
            center;"><b>Data Cleaning</b>

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Handling Missing Values</b>

In [9]:
# Check for missing values in the combined transactions

combined_transactions.isnull().sum()

authorized_flag               0
card_id                       0
city_id                       0
category_1                    0
installments                  0
category_3               234081
merchant_category_id          0
merchant_id              164697
month_lag                     0
purchase_amount               0
purchase_date                 0
category_2              2764609
state_id                      0
subsector_id                  0
dtype: int64

'merchant_id', 'category_3' and, 'category_2' are Categorical Features, So i will impute the missing values in these three columns using the Most Frequent Value for each of them.

In [ ]:
# Calculating and Printing the Mode for 'Category_2', 'Category_3', and 'Merchant_id' columns

category_2_mode = combined_transactions['category_2'].mode()[0]
category_3_mode = combined_transactions['category_3'].mode()[0]
merchant_id_mode = combined_transactions['merchant_id'].mode()[0]

print(f"Most Common in category_2 : {category_2_mode}")
print(f"Most Common in category_3 : {category_3_mode}")
print(f"Most Common in merchant_id : {merchant_id_mode}")

In [ ]:
# Fill the missing values in 'mercahnt_id', 'category_2', and 'category_3' columns

combined_transactions['merchant_id'].fillna(merchant_id_mode, inplace=True)
combined_transactions['category_3'].fillna(category_3_mode, inplace=True)
combined_transactions['category_2'].fillna(category_2_mode, inplace=True)

In [ ]:
# Check for missing values in combined transactions after filling them

combined_transactions.isnull().sum()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Handling Duplicated Values</b>

In [ ]:
# Check for duplicate rows in combined transactions

combined_transactions.duplicated().sum()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Handling Outliers</b>

In [ ]:
# Generate descriptive statistics for numerical columns in combined transactions

combined_transactions.describe().T

In [ ]:
# Display information about outliers

col = ['installments', 'month_lag', 'purchase_amount','category_2']
OutliersInfo(combined_transactions, col)

In [ ]:
# Count unique value in 'installments' column

combined_transactions['installments'].value_counts()

'Installments' should be between 0–12 months, however we can see month value of -1 and 999 which may be some error or wrong entries

In [ ]:
# Count the number of rows where 'purchase_amount' is greater than 2 

combined_transactions[combined_transactions['purchase_amount'] > 2].count()

In [ ]:
# Drop the rows where 'purchase_amount' is greater than 2 

combined_transactions = combined_transactions.drop(combined_transactions[combined_transactions['purchase_amount'] > 2].index)
combined_transactions.shape

In [ ]:
# Drop rows where 'installments' is -1 or 999

Remove_Values = [-1,999]
combined_transactions = combined_transactions[~combined_transactions['installments'].isin(Remove_Values)]
combined_transactions.shape

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Handling Data Formatting</b>

In [ ]:
# Convert the 'Purchase_date' column to datatime Type

combined_transactions['purchase_date']  = pd.to_datetime(combined_transactions['purchase_date'], format='%Y-%m-%d %H:%M:%S')

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:180%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:800;
            text-align:
            center;"><b>Data Visualization</b>

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Create function for categorical features</b>

In [ ]:
colors = sns.cubehelix_palette(20,reverse = True, light= 0.01,dark = 0.5, gamma= 0.7)
palette_color  = sns.color_palette("RdBu",10)
sns.set_theme(style="whitegrid", palette=palette_color)

def bar_plot(counts, column, ax, orient='v'):

    if orient=='h':
        x_col = 'count'
        y_col = column
    else:
        x_col = column
        y_col = 'count'

    sns.barplot(data = counts, x=x_col, y=y_col, ax=ax, orient=orient)


    if orient == 'v':
        # Annotate each bar with its height (number of occurrences)
        for p in ax.patches:
            x_coor = p.get_x() + 0.5 * p.get_width()
            y_coor = p.get_height()
            hight = int(p.get_height())

            ax.annotate(hight,            # Text to be displayed (converted to int for formatting)
                        (x_coor, y_coor), # Coordinates of the annotation (x, y)
                        ha='center',      # Horizontal alignment of the text ('center' aligns it at the center of the x-coordinate)
                        va='bottom',      # Vertical alignment of the text ('bottom' aligns it at the bottom of the bar)
                        color='black'     # Color of the text
                        )
        ax.set_xlabel(column, weight = "bold",  fontsize = 14, labelpad = 20)
        ax.set_ylabel('Number of Occurrences', weight = "bold", fontsize = 14, labelpad = 20)

    ax.tick_params(axis = 'both', labelsize = 12)

    return ax

In [ ]:
def plot_categorical_feature(counts, figsize=(12,5)):
   column = counts.columns[0]
    
   # Create a figure with two subplots
   fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)

   # Bar chart
   bar_plot(counts, column, ax1)

   # Pie chart
   ax2.pie(counts['count'], labels=counts[column], autopct = '%1.1f%%')
   ax2.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
   ax2.legend(counts[column], loc="best") # Add legned with labels

   # Adjust layout
   plt.suptitle(f'Distribution of {column}', weight = "bold", fontsize = 16)
   plt.tight_layout()
   plt.show()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Authorized Flag</b>

In [ ]:
authorized_flag_counts = combined_transactions['authorized_flag'].value_counts().reset_index()
authorized_flag_counts

In [ ]:
plot_categorical_feature(authorized_flag_counts)

The data shows that most transactions (92%) are authorized which mean that the purchase or transaction went through without any issues, while only (8%) are unauthorized which indicates that the transaction was declined or not approved for some reason.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Top 25 Card ID</b>

In [ ]:
Card_ID_Counts = combined_transactions['card_id'].value_counts().head(25).sort_values().reset_index()
Card_ID_Counts

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(data=Card_ID_Counts,x='count',y='card_id' ,orient='h')
plt.title('Top 25 Card')
plt.xlabel('Count')
plt.ylabel('Card ID')
plt.show()

# The data indicates that a few cards dominate transaction activity:

1- One card (C_ID_3d3dfdc692) has 5,526 transactions—almost double next highest card.

2- The rest of the top 25 cards range between 1,380 and 2,897 transactions, gradually decreasing.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Top 25 Merchant ID</b>

In [ ]:
Merchant_ID_Counts = combined_transactions['merchant_id'].value_counts().head(25).sort_values().reset_index()
Merchant_ID_Counts

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(data=Merchant_ID_Counts,x='count',y='merchant_id' ,orient='h')
plt.title('Top 25 Merchant')
plt.xlabel('Count')
plt.ylabel('Merchant ID')
plt.show()

# The data shows a few big merchants handle most transactions:

1- One Merchants likely account for over a million transactions.

2- Other merchants in the top 25 handle between 200,000-450,000 transactions

3- The #1 merchant (M_ID_00a6ca8a8a) appears to be the most popular.

# What This Means:

1- Customers mostly shop at frequently-used stores, These could be supermarkets, department stores, or popular service providers

2- The pattern suggests people tend to stick with certain merchants regularly

3- Loyalty scores may depend heavily on these frequent merchant relationships

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Category 1</b>

In [ ]:
Category_1_Counts = combined_transactions['category_1'].value_counts().reset_index()
Category_1_Counts

In [ ]:
plot_categorical_feature(Category_1_Counts)

The distribution of 'Category 1' is notably imbalanced, with 93.1% of occurrences labeled as 'N' and only 6.9% labeled as 'Y', highlighting a substantial class imbalance that may affect model performance.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Installments</b>

In [ ]:
Installments_Counts = combined_transactions['installments'].value_counts().reset_index()
Installments_Counts

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(data=Installments_Counts,x='count',y='installments' ,orient='h')
plt.title('Distribution of Installments')
plt.xlabel('Count')
plt.ylabel('Months')
plt.show()

Most transactions occur with 0 or 1 installment, with a sharp decrease in frequency as the number of installments increases, suggesting that customers generally prefer to pay in full or with a single installment.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Category 3</b>

In [ ]:
Category_3_Counts = combined_transactions['category_3'].value_counts().reset_index()
Category_3_Counts

In [ ]:
plot_categorical_feature(Category_3_Counts)

Category 'A' (53.2%) and 'B' (40.7%) dominate, with 'C' being much less common (6.2%), indicating a strong preference towards categories 'A' and 'B'.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Month Lag</b>

In [ ]:
Month_Lag_Counts = combined_transactions['month_lag'].value_counts().reset_index()
Month_Lag_Counts

In [ ]:
plt.figure(figsize=(10,8))
sns.barplot(data=Month_Lag_Counts,x='count',y='month_lag' ,orient='h')
plt.title('Distribution of Month Lag')
plt.xlabel('Count')
plt.ylabel('Months')
plt.show()

Most activity (+2.5 M transactions each) happened in the 4 months before the reference date or the same purchase date (lag months = 0), while older transactions (months -13 to -5) and future ones (months 1-2) appear less frequently, Which mean that the data focuses heavily on recent purchases.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Category 2</b>

In [ ]:
Category_2_Counts = combined_transactions['category_2'].value_counts().reset_index()
Category_2_Counts

In [ ]:
plot_categorical_feature(Category_2_Counts)

Category 2 is highly imbalanced, with the majority value being 1.0 (61.2%), while other categories are much less frequent.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Top 25 Card Purchased</b>

In [ ]:
grouped = combined_transactions.groupby('card_id')['purchase_amount'].sum().reset_index().sort_values(by='purchase_amount',ascending=False).head(25)
grouped

In [ ]:
top_25_card_Purchased = grouped['card_id'].tolist()
filtered_trans = combined_transactions[combined_transactions['card_id'].isin(top_25_card_Purchased)]
filtered_trans

In [ ]:
fig = px.box(filtered_trans,x='card_id',y='purchase_amount')

font_config = {
            'family': 'Arial',                # Font family
            'size': 24,                       # Font size
            'color': 'Black'          # Font color
        }

title_config = {
        'text': "Purchase Amount For Top 25 Card",               # The title text
        'x': 0.5,                             # x-position of the title (0 to 1, where 0.5 is centered)
        'xanchor': 'center',                  # Anchor point for the title's x position
        'y': 0.95,                            # y-position of the title (0 to 1)
        'yanchor': 'top',                     # Anchor point for the title's y position
        'font': font_config
        }

xaxis_config={
        'title': 'Card ID',                              # Axis title text
        'titlefont': {'size': 16, 'color': 'DarkBlue'}, # Font properties for the axis title
        'tickangle': -45,                                    # Angle of tick labels (degrees)
        'gridcolor': 'LightGray',                            # Grid line color
    }

yaxis_config={
        'title': 'Purchase Amount',
        'titlefont': {'size': 16, 'color': 'DarkBlue'},
    }

fig.update_layout(title= title_config,xaxis=xaxis_config, yaxis=yaxis_config)
fig.show()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Most Purchased Month</b>

In [ ]:
Most_Purchased_Month = combined_transactions.groupby(combined_transactions['purchase_date'].dt.month)['card_id'].count().reset_index()
Most_Purchased_Month

In [ ]:
sns.lineplot(data=Most_Purchased_Month,x='purchase_date',y='card_id')
plt.title('Transactions Over Months')
plt.xlabel('Month')
plt.ylabel('Count of Transactions')
plt.show()

There is a notable seasonal trend: transactions are highest at the beginning and end of the year and reach their lowest point around month 5, followed by a steady increase towards the end of the year.

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Distribution of Purchased Amount</b>

In [ ]:
sns.histplot(data=combined_transactions,x='purchase_amount',kde=True,bins=20)
plt.title('Distribution of Purchased Amount')
plt.xlabel('Purchased Amount')
plt.show()

# The purchase amounts show some clear trends:

1- Most purchases are small - The vast majority (20M+ transactions) are tiny amounts near zero

2- Big purchases are rare

3- Some negative values exist - These likely represent refunds or returns

# Why This Matters for Loyalty:

1- Customers mainly make small, frequent purchases

2- The occasional big spenders might be High-value customers or People making special purchases

3- Negative amounts (refunds) could indicate unhappy customers

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>KDE plot for Purchase Date</b>

In [ ]:
sns.displot(combined_transactions['purchase_date'].dt.year,kind='kde')
plt.title('Kde plot for purchase date')
plt.show()

The data shows two distinct shopping times Major Spike - Jan 2017 (very high), Smaller Spike - Jan 2018 (moderate), Dead Zone - Almost no activity between these peaks, and Customer behavior might have changed between 2017-2018, this could affect model accuracy

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:180%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:800;
            text-align:
            center;"><b>Data Preprocessing</b>

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Feature Encoding</b>

In [ ]:
# Encode categorical features to numerical 

combined_transactions['authorized_flag'] = combined_transactions['authorized_flag'].map({'Y': 1 , 'N': 0})
combined_transactions['category_1'] = combined_transactions['category_1'].map({'Y': 1 , 'N': 0})
combined_transactions['category_3'] = combined_transactions['category_3'].map({'A': 1 , 'B': 2 , 'C': 3})

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Feature Engineering on Purchase Date and Month Lag</b>

I will add five new columns in the transactions_df dataframe: purchase_year, purchase_month, purchase_day,  purchase_dow, and purchase_hour which contain the year, month, day, day of the week, and hour for each transaction, respectively. The dayofweek attribute returns the day of the week as an integer, where Monday is 0 and Sunday is 6. Note that we have also converted the year and month columns to categorical data type, as they are categorical variables. Steps:
<ol>
 <li>Converting the purchase_date column to a datetime object.</li>
 <li>Creating new columns for weekday, weekend indicator, month, day of year, week of year, hour, and holiday indicator.</li>
 <li>Calculating the number of months since each transaction using month_diff.</li>
</ol>

In [ ]:
# Extract the year from 'purchase_date' column and create 'purchase_year' column

combined_transactions['purchase_year'] = combined_transactions['purchase_date'].dt.year

In [ ]:
# Extract the month from 'purchase_date' column and create 'purchase_month' column


combined_transactions['purchase_month'] = combined_transactions['purchase_date'].dt.month

In [ ]:
# Extract the day from 'purchase_date' column and create 'purchase_day' column


combined_transactions['purchase_day'] = combined_transactions['purchase_date'].dt.day

In [ ]:
# Extract the day of week from 'purchase_date' column and create 'purchase_dow' column


combined_transactions['purchase_dow'] = combined_transactions['purchase_date'].dt.day_of_week

In [ ]:
# Extract the hour from 'purchase_date' column and create 'purchase_hour' column


combined_transactions['purchase_hour'] = combined_transactions['purchase_date'].dt.hour

In [ ]:
# Create 'is_weekend' column: 1 if 'purchase_dow' is Saturday (5) or Sunday (6), 0 otherwise

combined_transactions['is_weekend'] = np.where(combined_transactions['purchase_dow'].isin([5, 6]), 1, 0)

In [ ]:
# Define time intervals based on the 'purchase_hour' column to Create 'purchase_at' column

time_day = [
    (combined_transactions['purchase_hour'] >= 6) & (combined_transactions['purchase_hour'] < 12),
    (combined_transactions['purchase_hour'] >= 12) & (combined_transactions['purchase_hour'] < 15),
    (combined_transactions['purchase_hour'] >= 15) & (combined_transactions['purchase_hour'] < 18),
    (combined_transactions['purchase_hour'] >= 18) & (combined_transactions['purchase_hour'] < 24)
]

In [ ]:
# Define labels for different time periods of the day

purchase_at = ['Morning', 'Noon', 'Afternoon', 'Evening']

# Create 'purchase_at' column based on 'time_day' conditions and 'purchase_at' labels

combined_transactions['purchase_at'] = np.select(time_day, purchase_at, default='Night')

In [ ]:
# Get today's date

today = np.datetime64(datetime.today())

In [ ]:
# Calculate the difference in days between today and 'purchase_date'

days_difference = (today - combined_transactions['purchase_date'].values).astype('timedelta64[D]').astype(int)

In [ ]:
# Calculate the months difference, assuming 30 days per month

combined_transactions['month_difference'] = (days_difference // 30).astype('int16')

In [ ]:
# Adjust 'month_difference' by substracting 'month_lag'

combined_transactions['month_difference'] = combined_transactions['month_difference'] - combined_transactions['month_lag']

In [ ]:
# Convert 'purchase_date' column to datetime objects

combined_transactions['purchase_date'] = pd.to_datetime(combined_transactions['purchase_date'])

In [ ]:
# Display the first rows of the Data

pd.set_option('display.max_columns', None)
combined_transactions.head()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>First Features Aggregation</b>

In [ ]:
# Create Aggregation dictionary to define how to aggregate columns

aggregate_tbl = {
    'purchase_date':['min','max'],
    'purchase_year': [statistics.mode],
    'purchase_month': ['mean',statistics.mode],
    'purchase_day': [statistics.mode],
    'purchase_dow': [statistics.mode],
    'purchase_hour': ['min','max',statistics.mode],
    'is_weekend': ['mean','sum',statistics.mode],
    'month_difference': ['sum','mean','min','max'] 
    }


In [ ]:
# Group the 'combined_transactions' by 'card_id' and apply the aggregations 

aggregate_purchase_date = combined_transactions.groupby('card_id').agg(aggregate_tbl).reset_index()
aggregate_purchase_date.head()

In [ ]:
# Rename the columns of the aggregated Dataframe with 'trans_' prefix and concatenated column names

aggregate_purchase_date.columns = ['trans_'+'_'.join(col).strip() for col in aggregate_purchase_date.columns.values]
aggregate_purchase_date = aggregate_purchase_date.rename(columns={'trans_card_id_':'card_id'})
aggregate_purchase_date.head()

In [ ]:
# Calculate the number of days since the first and last transactions for each card_id 

aggregate_purchase_date['first_transaction'] = (datetime.today() - aggregate_purchase_date['trans_purchase_date_min']).dt.days
aggregate_purchase_date['last_transaction'] = (datetime.today() - aggregate_purchase_date['trans_purchase_date_max']).dt.days

In [ ]:
# Intialize OneHotEncoder to convert categorical 'purchase_at' to numerical feature

encoder = OneHotEncoder()
one_hot = encoder.fit_transform(combined_transactions[['purchase_at']])

In [ ]:
# Create Dataframe from the one-hot encoded array with feature names

df = pd.DataFrame(one_hot.toarray(),columns=encoder.get_feature_names_out())
df.head()

In [ ]:
# Reset the index of combined_transactions
combined_transactions = combined_transactions.reset_index(drop=True)

#Reset index of df
df = df.reset_index(drop=True)

# Concatenate the 'combined_transactions' DataFrame with the one-hot encoded DataFrame
combined_transactions = pd.concat([combined_transactions, df], axis=1)
combined_transactions.head()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Feature Engineering and Aggregation on Purchase Amount</b>

Computes several aggregate statistics ontransactions the purchase_amount column for each group. The computed statistics are the minimum, maximum, mean, median, standard deviation, sum, and count.This aggregated data gives an overview of each customer's spending behavior:

<ul>
 <li>High maximum or average amounts indicate big ticket purchases</li>
 <li>A wide spread (high standard deviation) suggests irregular spending</li>
 <li>A high total spend sum purchase_amount captures overall expenditures</li>
 <li>The count of transactions reveals purchase frequency</li>
 </ul>

In [ ]:
# Group the 'combined_transactions' DataFrame by 'card_id' and aggregate 'purchase_amount'

aggregate_purchase_amount = combined_transactions.groupby('card_id')['purchase_amount'].agg(['sum','max','min','mean','median']).reset_index()
aggregate_purchase_amount.head()

In [ ]:
# Rename the columns of the 'aggregate_purchase_amount' DataFrame

aggregate_purchase_amount.columns = ['trans_purchase_amount_'+''.join(col).strip() for col in aggregate_purchase_amount.columns.values]
aggregate_purchase_amount = aggregate_purchase_amount.rename(columns={'trans_purchase_amount_card_id':'card_id'})
aggregate_purchase_amount.head()

<div style="padding:10px;
            color:#E55604;
            margin:10px;
            font-size:130%;
            display:fill;
            border-radius:5px;
            border-style: solid;
            border-color: #FF9B50;
            background-color:#000000;
            overflow:hidden;
            font-weight:400"><b>Third Features Aggregation</b>

In [ ]:
# Define an aggregation dictionary

aggregate_tbl2 = {
    'authorized_flag': ['sum','mean'],
    'city_id': [statistics.mode,'nunique'],
    'category_1': ['sum','mean',statistics.mode],
    'category_2': ['sum','mean',statistics.mode],
    'category_3': ['sum','mean',statistics.mode],
    'installments': ['mean','sum','min','max',statistics.mode],
    'merchant_category_id': [statistics.mode,'nunique'],
    'merchant_id': [statistics.mode,'nunique'],
    'state_id': [statistics.mode,'nunique'],
    'subsector_id': ['nunique',statistics.mode],
    'purchase_at_Afternoon': [statistics.mode,'sum','mean'],
    'purchase_at_Evening': [statistics.mode,'sum','mean'],
    'purchase_at_Morning': [statistics.mode,'sum','mean'],
    'purchase_at_Night': [statistics.mode,'sum','mean'],
    'purchase_at_Noon': [statistics.mode,'sum','mean']
}

In [ ]:
# Dropping 'purchase_at' column

combined_transactions = combined_transactions.drop(columns='purchase_at',axis=1)

In [ ]:
# Convert category columns to object

combined_transactions['category_1'] = combined_transactions['category_1'].astype('object')
combined_transactions['category_3'] = combined_transactions['category_3'].astype('object')

In [ ]:
# Group 'combined_transactions' by 'card_id' and apply aggregations defined 

aggregate_features = combined_transactions.groupby('card_id').agg(aggregate_tbl2).reset_index()
aggregate_features.head()

In [ ]:
# Rename the columns of the 'aggregate_features' DataFrame

aggregate_features.columns = [str(uppers)+'_'+str(lowers) for uppers, lowers in aggregate_features.columns.values]
aggregate_features.head()

In [ ]:
# Concatenate 'aggregate_purchase_date', 'aggregate_purchase_amount' and 'aggregate_features'

aggregate_transactions = pd.concat([aggregate_purchase_date,aggregate_purchase_amount,aggregate_features],axis=1)

In [ ]:
# Display the number of rows and columns of 'aggregate_transactions'

aggregate_transactions.shape

In [ ]:
# Checking the number of null values in the 'aggregate_transactions' DataFrame

aggregate_transactions.isnull().sum().sum()

In [ ]:
# Perform garbage collection to free up memory 

gc.collect()

In [ ]:
# Save the 'aggregate_transactions' DataFrame to CSV file without the Index
aggregate_transactions.to_csv('/kaggle/working/Aggregate_Transactions.csv', index=False)

print("aggregate_transactions saved to /kaggle/working/aggregate_transactions.csv")